# Downlaods smartsheets using API
* imports libraies as well

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import smartsheet

#Variables
# random
excel = '.xlsx'
csv = '.csv'

# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

#download smartsheets to any folder you need
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 


# #Google Sheets
# #Sheet ID comes right after /d/ in the URL
# Google_sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# # name of tab you want to use
# Google_sheets_name = 'Tree RTR Tracker'

# # this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
# url = f'https://docs.google.com/spreadsheets/d/{Google_sheet_ID}/export?format=csv#id={Google_sheets_name}'



# Function to read in column titles and row vales and make out data frames from smartsheets
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

# ROE transmittal
* combines all files in one folder
* will have to manually download each transmittal untill further notice

In [6]:
joined_files = os.path.join(r"C:\Users\jacque.trahan\Desktop\ROE checks", "*.xlsx")
joined_list = glob.glob(joined_files)


df = pd.concat(map(pd.read_excel, joined_list), ignore_index=True)

C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\U

In [9]:
ROE_t = df[['APN', 'Street #', 'Street Name']].copy()
SS = df_both[['APN_ROW Segment','Street #', 'Street Name']].copy()


In [10]:
Check = SS.merge(ROE_t,
                left_on='APN_ROW Segment',
                right_on='APN',
                how='outer',
                suffixes=("_SS","_ROE"))

In [17]:
Check['Street #_SS'] = Check['Street #_SS'].astype(str)
Check['Street Name_SS'] = Check['Street Name_SS'].astype(str)
Check['Street #_ROE'] = Check['Street #_ROE'].astype(str)
Check['Street Name_ROE'] = Check['Street Name_ROE'].astype(str)

In [18]:
Check['SmartSheet address'] = Check['Street #_SS'] + " " + Check['Street Name_SS']
Check['ROE address'] = Check['Street #_ROE'] + " " + Check['Street Name_ROE']

In [21]:
Checks = Check[Check['SmartSheet address'] != Check['ROE address']]

In [23]:
Checks.to_excel('ROE Adress check.xlsx')